## Movie Recommendation System Project 

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [3]:
movies=pd.read_csv("D:\\archive\\top10K-TMDB-movies.csv")

#### Analyzing our data

In [4]:
movies.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [5]:
movies.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.3+ KB


In [7]:
movies.isnull().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [8]:
movies.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')


#### Feature selection


In [9]:
movies['info']=movies["overview"]+movies["genre"]
movies_selected_features=movies.drop(["popularity",'vote_count','vote_average','genre','overview'],axis=1)
movies_selected_features.columns

Index(['id', 'title', 'original_language', 'release_date', 'info'], dtype='object')

In [10]:
movies_selected_features.head()

,id,title,original_language,release_date,info
0,278,The Shawshank Redemption,en,1994-09-23,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,hi,1995-10-19,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,en,1972-03-14,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,en,1993-12-15,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,en,1974-12-20,In the continuing saga of the Corleone crime f...


#### Using CountVectorizer from Sklearn

In [11]:
cv=CountVectorizer(max_features=10000,stop_words='english')

In [12]:
cv

CountVectorizer(max_features=10000, stop_words='english')

In [13]:
# Fit data to CountVectorizer()
vector=cv.fit_transform(movies_selected_features['info'].values.astype('U')).toarray()

In [14]:
vector.shape

(10000, 10000)

#### Import `cosine_similarity` from `sklearn.metrics`

In [15]:
similarity=cosine_similarity(vector)

In [16]:
similarity

array([[1.        , 0.05634362, 0.12888482, ..., 0.07559289, 0.11065667,
        0.06388766],
       [0.05634362, 1.        , 0.07624929, ..., 0.        , 0.03636965,
        0.        ],
       [0.12888482, 0.07624929, 1.        , ..., 0.02273314, 0.06655583,
        0.08645856],
       ...,
       [0.07559289, 0.        , 0.02273314, ..., 1.        , 0.03253   ,
        0.02817181],
       [0.11065667, 0.03636965, 0.06655583, ..., 0.03253   , 1.        ,
        0.0412393 ],
       [0.06388766, 0.        , 0.08645856, ..., 0.02817181, 0.0412393 ,
        1.        ]])

#### Accessing the index number for the system

In [17]:
movies_selected_features[movies_selected_features['title']=='The Godfather'].index[0]

2

In [18]:
recom_sys=sorted(list(enumerate(similarity[2])),reverse=True,key=lambda vector:vector[1])
for i in recom_sys[0:5]:
    print(movies_selected_features.iloc[i[0]].title)

The Godfather
The Godfather: Part II
Blood Ties
Joker
Bomb City


#### Building a recommend function

In [19]:
def recommend(movies):
    """
    Recommends Movies Based on the input of the user
    """
    index=movies_selected_features[movies_selected_features['title']==movies].index[0]
    recom_sys=sorted(list(enumerate(similarity[index])),reverse=True,key=lambda vector:vector[1])
    for i in recom_sys[0:5]:
        print(movies_selected_features.iloc[i[0]].title)

In [20]:
recommend('The Godfather')

The Godfather
The Godfather: Part II
Blood Ties
Joker
Bomb City


#### Saving the model

In [21]:
pickle.dump(movies_selected_features,open('movies_list.pkl','wb'))

In [22]:
pickle.dump(similarity,open('similarity.pkl','wb'))

#### Load the model

In [45]:
pickle.load(open('movies_list.pkl','rb'))

,id,title,original_language,release_date,info
0,278,The Shawshank Redemption,en,1994-09-23,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,hi,1995-10-19,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,en,1972-03-14,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,en,1993-12-15,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,en,1974-12-20,In the continuing saga of the Corleone crime f...
...,...,...,...,...,...
9995,10196,The Last Airbender,en,2010-06-30,"The story follows the adventures of Aang, a yo..."
9996,331446,Sharknado 3: Oh Hell No!,en,2015-07-22,The sharks take bite out of the East Coast whe...
9997,13995,Captain America,en,1990-12-14,"During World War II, a brave, patriotic Americ..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,en,2007-11-29,A man named Farmer sets out to rescue his kidn...
